## Imports and model initialization

In [1]:
# !pip install kipoi
# !pip install kipoiseq
# !pip install pybedtools
# !pip uninstall -y kipoi_veff
# !pip install git+https://github.com/an1lam/kipoi-veff
# !pip install pyvcf
import kipoi
from kipoi_interpret.importance_scores.ism import Mutation
from kipoiseq.dataloaders import SeqIntervalDl
import numpy as np

In [38]:
!pwd

/home/stephenmalina/project/src


In [2]:
import tensorflow as tf
print("TF version:", tf.__version__)
import torch
print("torch version:", torch.__version__)

/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passi

TF version: 1.14.0
torch version: 1.3.1


In [3]:
# df = kipoi.list_models()
# deepsea_models = df[df.model.str.contains("DeepSEA")]
# deepsea_models.head()

In [4]:
deepsea = kipoi.get_model("DeepSEA/variantEffects", source="kipoi")

Using downloaded and verified file: /home/jupyter/.kipoi/models/DeepSEA/variantEffects/downloaded/model_files/weights/35956ab9c28960b5a3693f470fe980c1


# Loading DNA sequence data

In [5]:
dl = SeqIntervalDl("../dat/ChIPseq.A549.CTCF.1000.random.narrowPeak.gz", "../dat/hg19.fa", auto_resize_len=1000)
data = dl.load_all()

100%|██████████| 32/32 [00:03<00:00,  8.84it/s]


In [6]:
seqs = np.expand_dims(data['inputs'].transpose(0, 2, 1), 2).astype(np.float32)
seqs.shape

(1001, 4, 1, 1000)

# Hacking the model
This section is focused on tweaking DeepSEA's model to use dropout when making predictions.

In [9]:
deepsea.model

Sequential(
  (0): ReCodeAlphabet()
  (1): Sequential(
    (0): Conv2d(4, 320, kernel_size=(1, 8), stride=(1, 1))
    (1): Threshold(threshold=0, value=1e-06)
    (2): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.2, inplace=False)
    (4): Conv2d(320, 480, kernel_size=(1, 8), stride=(1, 1))
    (5): Threshold(threshold=0, value=1e-06)
    (6): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.2, inplace=False)
    (8): Conv2d(480, 960, kernel_size=(1, 8), stride=(1, 1))
    (9): Threshold(threshold=0, value=1e-06)
    (10): Dropout(p=0.5, inplace=False)
    (11): Lambda()
    (12): Sequential(
      (0): Lambda()
      (1): Linear(in_features=50880, out_features=925, bias=True)
    )
    (13): Threshold(threshold=0, value=1e-06)
    (14): Sequential(
      (0): Lambda()
      (1): Linear(in_features=925, out_features=919, bias=True)
    )
    (15): Sigmoid()
  )
)

In [28]:
from torch import nn

def apply_dropout(m):
    if type(m) == nn.Dropout:
        m.train()
        
def unapply_dropout(m):
    if type(m) == nn.Dropout:
        m.eval()

### Verifying dropout is being applied
We can verify or at least get reasonable certainty that dropout's being applied by running 10 predictions on the same sequence and then seeing that the predicted binding probs differ. First, we show that the predictions stay the same if dropout is off.

Then, we turn on dropout and show that they start to vary.

In [ ]:
preds = deepsea.pipeline.predict_example()
for i in range(3):
    print(f"First few preds ({i+1}/3): ", deepsea.pipeline.predict_example()[:5])

In [ ]:
deepsea.model.eval()
deepsea.model = deepsea.model.apply(apply_dropout)
preds = deepsea.pipeline.predict_example()
for i in range(3):
    print(f"First few preds ({i+1}/3): ", deepsea.pipeline.predict_example()[:5])

## Predictions and in-silico mutagenesis

In [7]:
CHROM_ACC_COL = 'A549_DNase_None'
TF_COL = 'A549_CTCF_None'
relevant_cols = sorted([(i, label)
                        for i, label in enumerate(deepsea.schema.targets.column_labels)
                        if label in [CHROM_ACC_COL, TF_COL]])

def output_sel_fn(result):
    return np.array([result[col_idx] for col_idx, _ in relevant_cols])

relevant_cols

[(52, 'A549_DNase_None'), (720, 'A549_CTCF_None'), (763, 'A549_CTCF_None')]

In [8]:
ism = Mutation(deepsea, "seq", scores=['ref', 'diff'], output_sel_fn=output_sel_fn)
ism_score = np.array(ism.score(seqs))

KeyboardInterrupt: 

In [ ]:
np.squeeze(ism_score).shape

In [ ]:
zeros = np.zeros((2, 3))
def sanitize_scores(scores):
    orig_shape = scores.shape
    sanitized_scores = np.ndarray((*orig_shape, 2, 3), dtype=scores.dtype)
    flattened_scores = scores.reshape(-1)
    
    for i, score in enumerate(flattened_scores):
        idx = np.unravel_index(i, orig_shape)
        if score is None: sanitized_scores[idx] = zeros
        else: sanitized_scores[idx] = np.array(score)
    return sanitized_scores

sanitized_scores = sanitize_scores(np.squeeze(ism_score))
ctcf_original_preds = sanitized_scores[:, :, :, 0, 1]
ctcf_pred_diffs = sanitized_scores[:, :, :, 1, 1]
dnase_original_preds = sanitized_scores[:, :, :, 0, 0]
dnase_pred_diff = sanitized_scores[:, :, :, 1, 0]

In [ ]:
import math
import numpy as np

log_uniform_prop = math.log(.05/(1-.05))
def compute_normalized_prob(prob, train_prob):
    denom = 1+np.exp(-(np.log(prob/(1-prob))+log_uniform_prop-np.log(train_prob/(1-train_prob))))
    return 1 / denom

# Ratios and normalization formula drawn from here: http://deepsea.princeton.edu/media/help/posproportion.txt
tf_compute_normalized_prob = lambda prob: compute_normalized_prob(prob, .020029)
# ENCODE	A549	DNase	None	0.048136
chrom_acc_normalized_prob = lambda prob: compute_normalized_prob(prob, 0.048136)

In [ ]:
IDX_TO_NT = 'ACGT'

def _convert_to_mutation(pos_nt_pair):
    return "%d%s" % (pos_nt_pair[1], IDX_TO_NT[pos_nt_pair[0]])

def find_best_mutation(mutation_map, sign=1):
    seq_len = mutation_map.shape[1]
    best_mute = None
    best_mute_effect = 0
    second_best_mute = None
    second_best_mute_effect = 0
    for seq_idx in range(seq_len):  # iterate over sequence
        best_out_of_nts_mute = None
        best_out_of_nts_mute_effect = 0 
        for nt_idx in range(4):  # iterate over nucleotides
            current_effect = mutation_map[nt_idx, seq_idx]
            
            if sign * current_effect > sign * best_out_of_nts_mute_effect:
                best_out_of_nts_mute_effect = current_effect
                best_out_of_nts_mute = (nt_idx, seq_idx)

        # TODO(Stephen): the right way to do this is to have a heap, squish the 2D array into a 1D
        # array of (effect, (seq_idx, nt_idx)) tuples and then take the top-k from the heap, but
        # I'm too lazy to do this right now.
        if sign * best_out_of_nts_mute_effect > (sign * best_mute_effect):
            best_mute_effect = best_out_of_nts_mute_effect
            best_mute = best_out_of_nts_mute
        elif sign * best_out_of_nts_mute_effect > (sign * second_best_mute_effect):
            second_best_mute_effect = best_out_of_nts_mute_effect
            second_best_mute = best_out_of_nts_mute 
    print(best_mute, best_mute_effect, second_best_mute, second_best_mute_effect)
    return [best_mute, second_best_mute]


import csv

mut_effects_fpath = "../dat/mut_effects_1.csv"
with open(mut_effects_fpath, 'w', newline="") as out_file:
    fieldnames = [
        "initial X prediction",
        "new X prediction",
        "initial Y prediction",
        "new Y prediction",
        "mutation",
    ]
    writer = csv.DictWriter(out_file, delimiter=",", fieldnames=fieldnames)
    writer.writeheader()
    
    for i, ctcf_pred_diff in enumerate(ctcf_pred_diffs):
        best_mut_idx, best_mut_effect = find_best_mutation(ctcf_pred_diff, sign=-1)
        nt_idx, seq_idx = best_mut_idx
        ctcf_original_pred = ctcf_original_preds[i][nt_idx, seq_idx]
        ctcf_new_pred = ctcf_original_pred + ctcf_pred_diff[nt_idx, seq_idx]
        dnase_original_pred = dnase_original_preds[i][nt_idx, seq_idx]
        dnase_pred_diff = dnase_pred_diffs[i][nt_idx, seq_idx]
        dnase_new_prd = dnase_original_pred + dnase_pred_diff
        writer.writerow(
            {
                "initial X prediction": ctcf_original_pred,
                "new X prediction": ctcf_new_pred,
                "initial Y prediction": dnase_original_pred,
                "new Y prediction": dnase_new_pred,
                "mutation": _convert_to_mutation(best_mut_idx),
            }
        )
        
